In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="qz3gXKPbOfhvVaI8oDt4")
project = rf.workspace("rkm-nnbdx").project("microfocus")
version = project.version(3)
dataset = version.download("yolov7")
                

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 33.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.7 MB/s eta 0:00:0000:01
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sigstore 3.6.1 requires rich~=13.0, but you have rich 14.0.0 which is incompatible.
datasets 3.5.0 requires fsspec[http]<=2024.


Extracting Dataset Version Zip to Microfocus-3 in yolov7pytorch:: 100%|██████████| 1992/1992 [00:00<00:00, 6518.01it/s]


**18th April**

In [3]:
# -----------------------------
# 1) Convert YOLOv11 polygons → PNG masks
#    (unchanged)
# -----------------------------
import os
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm

base = dataset.location   # e.g. "/content/Microfocus-3"
IMG_SIZE = (256, 256)     # width, height

def yolo11_poly_to_mask(img_path: str, label_path: str, img_size=IMG_SIZE):
    w, h = img_size
    mask = np.zeros((h, w), dtype=np.uint8)
    if os.path.exists(label_path):
        for line in open(label_path):
            parts = list(map(float, line.strip().split()))
            # first entry is class id
            coords = parts[1:]
            pts = np.array(coords).reshape(-1, 2)
            pts[:,0] *= w; pts[:,1] *= h
            cv2.fillPoly(mask, [pts.astype(np.int32)], 255)
    return mask

def create_masks_for_split(split: str):
    img_dir = os.path.join(base, split, "images")
    lbl_dir = os.path.join(base, split, "labels")
    msk_dir = os.path.join(base, split, "masks")
    os.makedirs(msk_dir, exist_ok=True)
    for img_path in tqdm(glob(os.path.join(img_dir, "*.jpg")), desc=f"{split}→masks"):
        fn    = os.path.basename(img_path).rsplit(".",1)[0]
        label = os.path.join(lbl_dir, fn + ".txt")
        mask  = yolo11_poly_to_mask(img_path, label, IMG_SIZE)
        cv2.imwrite(os.path.join(msk_dir, fn + ".png"), mask)

# regenerate masks
create_masks_for_split("train")
create_masks_for_split("valid")
create_masks_for_split("test")  # optional

# -----------------------------
# 2) Build tf.data pipelines w/ augmentation
# -----------------------------
import tensorflow as tf

BATCH_SIZE = 8
AUTOTUNE   = tf.data.AUTOTUNE

def load_image_and_mask(img_path, mask_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE) / 255.0

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.resize(mask, IMG_SIZE,
                           method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    mask = tf.cast(mask > 127, tf.float32)
    return img, mask

def augment(img, mask):
    # random flips
    if tf.random.uniform(()) > 0.5:
        img = tf.image.flip_left_right(img)
        mask = tf.image.flip_left_right(mask)
    if tf.random.uniform(()) > 0.5:
        img = tf.image.random_flip_up_down(img)
        mask = tf.image.random_flip_up_down(mask)
    # random brightness
    img = tf.image.random_brightness(img, max_delta=0.1)
    return img, mask

def make_ds(split: str):
    img_dir  = os.path.join(base, split, "images")
    mask_dir = os.path.join(base, split, "masks")
    imgs  = sorted(glob(os.path.join(img_dir, "*.jpg")))
    masks = [os.path.join(mask_dir, os.path.basename(p).replace(".jpg", ".png")) 
             for p in imgs]

    ds = tf.data.Dataset.from_tensor_slices((
        tf.constant(imgs,  dtype=tf.string),
        tf.constant(masks, dtype=tf.string)
    ))
    ds = ds.map(load_image_and_mask, num_parallel_calls=AUTOTUNE)
    if split == "train":
        ds = ds.map(augment, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return ds

train_ds = make_ds("train")
val_ds   = make_ds("valid")
test_ds  = make_ds("test")

# -----------------------------
# 3) Define combined BCE + Dice loss
# -----------------------------
import tensorflow.keras.backend as K

def dice_loss(y_true, y_pred):
    smooth = 1e-6
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    return bce + dice_loss(y_true, y_pred)

# -----------------------------
# 4) Build ResNet‑Attention U‑Net with Dropout
# -----------------------------
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, \
    UpSampling2D, Add, Multiply, Concatenate, Dropout, Input
from tensorflow.keras.models import Model

def conv_block(x, filters):
    x = Conv2D(filters, 3, padding="same", activation="relu")(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters, 3, padding="same", activation="relu")(x)
    x = BatchNormalization()(x)
    return x

def attention_block(skip, gate, inter_channels):
    theta_x = Conv2D(inter_channels, 1, padding="same")(skip)
    phi_g   = Conv2D(inter_channels, 1, padding="same")(gate)
    # upsample phi_g to skip size
    skip_h, skip_w = K.int_shape(skip)[1:3]
    gate_h, gate_w = K.int_shape(phi_g)[1:3]
    scale_h = skip_h // gate_h
    scale_w = skip_w // gate_w
    phi_g_up = UpSampling2D(size=(scale_h, scale_w), interpolation="bilinear")(phi_g)
    add    = Add()([theta_x, phi_g_up])
    act    = Activation("relu")(add)
    psi    = Conv2D(1, 1, padding="same", activation="sigmoid")(act)
    attn   = Multiply()([skip, psi])
    return attn

def build_res_attention_unet(input_shape=(256,256,3)):
    inputs     = Input(shape=input_shape)
    base_model = tf.keras.applications.ResNet50(
        include_top=False, weights="imagenet", input_tensor=inputs
    )
    skips = [base_model.get_layer(name).output for name in [
        "conv1_relu", "conv2_block3_out", "conv3_block4_out", "conv4_block6_out"
    ]]
    x = base_model.get_layer("conv5_block3_out").output

    filters = [512, 256, 128, 64]
    for i, skip in enumerate(reversed(skips)):
        x = UpSampling2D(size=(2,2), interpolation="bilinear")(x)
        attn = attention_block(skip, x, inter_channels=filters[i]//2)
        x = Concatenate()([x, attn])
        x = conv_block(x, filters[i])
        x = Dropout(0.3)(x)            # <-- dropout to regularize

    # final upsample
    x = UpSampling2D(size=(2,2), interpolation="bilinear")(x)
    x = conv_block(x, 32)
    x = Dropout(0.3)(x)
    outputs = Conv2D(1, 1, activation="sigmoid")(x)
    return Model(inputs, outputs, name="ResNet_Att_UNet")

model = build_res_attention_unet((IMG_SIZE[1], IMG_SIZE[0], 3))

# -----------------------------
# 5) Compile with new loss & metrics
# -----------------------------
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=bce_dice_loss,
    metrics=[
        "accuracy",
        tf.keras.metrics.MeanIoU(num_classes=2, name="mean_iou")
    ]
)
model.summary()

# -----------------------------
# 6) Callbacks (tighter patience)
# -----------------------------
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        "resattunet_best.keras",
        monitor="val_loss", save_best_only=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=5, verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    ),
]

# -----------------------------
# 7) Train
# -----------------------------
EPOCHS = 100
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=True
)


test→masks: 100%|██████████| 50/50 [00:00<00:00, 1918.36it/s]
2025-04-18 04:07:06.518078: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744949226.768778      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744949226.838910      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1744949240.354513      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1744949240.355297      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1,

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "ResNet_Att_UNet"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 262, 262, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 128, 128, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 128, 128, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 128, 128, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 130, 130, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 64, 64, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 64, 64, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 64, 64, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 64, 64, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 64, 64, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 64, 64, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 64, 64, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 64, 64, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 64, 64, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 44,932,101 (171.40 MB)

...

Epoch 1/100


I0000 00:00:1744949305.369272     102 service.cc:148] XLA service 0x79f040002260 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744949305.370223     102 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1744949305.370233     102 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1744949311.150465     102 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1744949363.278158     102 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


106/107 ━━━━━━━━━━━━━━━━━━━━ 0s 361ms/step - accuracy: 0.6848 - loss: 1.3680 - mean_iou: 0.3811

E0000 00:00:1744949419.380447     102 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1744949419.681473     102 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


107/107 ━━━━━━━━━━━━━━━━━━━━ 240s 1s/step - accuracy: 0.6846 - loss: 1.3694 - mean_iou: 0.3804 - val_accuracy: 0.7786 - val_loss: 1.3535 - val_mean_iou: 0.4045 - learning_rate: 1.0000e-04
Epoch 2/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 46s 424ms/step - accuracy: 0.7701 - loss: 1.2232 - mean_iou: 0.3808 - val_accuracy: 0.8057 - val_loss: 1.3177 - val_mean_iou: 0.4045 - learning_rate: 1.0000e-04
Epoch 3/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 45s 419ms/step - accuracy: 0.8077 - loss: 1.1565 - mean_iou: 0.3814 - val_accuracy: 0.8089 - val_loss: 1.3147 - val_mean_iou: 0.4045 - learning_rate: 1.0000e-04
Epoch 4/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 45s 422ms/step - accuracy: 0.8339 - loss: 1.1284 - mean_iou: 0.3809 - val_accuracy: 0.8089 - val_loss: 1.3045 - val_mean_iou: 0.4045 - learning_rate: 1.0000e-04
Epoch 5/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 45s 420ms/step - accuracy: 0.8543 - loss: 1.0707 - mean_iou: 0.3811 - val_accuracy: 0.8089 - val_loss: 1.3009 - val_mean_iou: 0.4045 - learning_rate: 1.0000e-04
Epoc

In [6]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# 1) Save your final model
model.save("resattunet_final.keras")

# Also save weights in HDF5 if you need legacy format:
# model.save_weights("resattunet_weights.h5")

# 2) Convert history to DataFrame and save CSV
hist_df = pd.DataFrame(history.history)
hist_df['epoch'] = hist_df.index + 1
hist_df.to_csv("training_history.csv", index=False)

# 3) Plot & save curves
os.makedirs("figures", exist_ok=True)

# Loss
plt.figure(figsize=(6,4))
plt.plot(hist_df['epoch'], hist_df['loss'],  label="Train Loss")
plt.plot(hist_df['epoch'], hist_df['val_loss'], label="Val Loss")
plt.xlabel("Epoch"); plt.ylabel("Loss")
plt.title("Training vs Validation Loss")
plt.legend(); plt.tight_layout()
plt.savefig("figures/loss_curve.png", dpi=300)
plt.close()

# Accuracy
plt.figure(figsize=(6,4))
plt.plot(hist_df['epoch'], hist_df['accuracy'],      label="Train Acc")
plt.plot(hist_df['epoch'], hist_df['val_accuracy'],  label="Val Acc")
plt.xlabel("Epoch"); plt.ylabel("Accuracy")
plt.title("Training vs Validation Accuracy")
plt.legend(); plt.tight_layout()
plt.savefig("figures/accuracy_curve.png", dpi=300)
plt.close()

# Mean IoU
plt.figure(figsize=(6,4))
plt.plot(hist_df['epoch'], hist_df['mean_iou'],      label="Train IoU")
plt.plot(hist_df['epoch'], hist_df['val_mean_iou'],  label="Val IoU")
plt.xlabel("Epoch"); plt.ylabel("Mean IoU")
plt.title("Training vs Validation Mean IoU")
plt.legend(); plt.tight_layout()
plt.savefig("figures/iou_curve.png", dpi=300)
plt.close()

# (Optional) Learning rate schedule if you want:
try:
    lrs = history.history['lr']
    plt.figure(figsize=(6,4))
    plt.plot(hist_df['epoch'], lrs, label="Learning Rate")
    plt.xlabel("Epoch"); plt.ylabel("LR")
    plt.title("Learning Rate Schedule")
    plt.legend(); plt.tight_layout()
    plt.savefig("figures/lr_schedule.png", dpi=300)
    plt.close()
except KeyError:
    pass  # lr not logged

# 4) Visualize & save sample predictions
def save_sample_predictions(dataset, model, num=5, thresh=0.5):
    import tensorflow as tf
    for imgs, masks in dataset.take(1):
        preds = model.predict(imgs)
        for i in range(min(num, imgs.shape[0])):
            fig, axes = plt.subplots(1,3, figsize=(12,4))
            axes[0].imshow(imgs[i]); axes[0].set_title("Input"); axes[0].axis('off')
            axes[1].imshow(tf.squeeze(masks[i]), cmap='gray')
            axes[1].set_title("Ground Truth"); axes[1].axis('off')
            axes[2].imshow((preds[i,...,0] > thresh).astype('float32'), cmap='jet')
            axes[2].set_title("Prediction"); axes[2].axis('off')
            plt.tight_layout()
            plt.savefig(f"figures/pred_{i+1}.png", dpi=300)
            plt.close()

save_sample_predictions(val_ds, model, num=5)

# 5) Final evaluate on test set
test_metrics = model.evaluate(test_ds, verbose=2)
print(f"Test Loss: {test_metrics[0]:.4f} | Test Acc: {test_metrics[1]:.4f} | Test IoU: {test_metrics[2]:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
7/7 - 1s - 115ms/step - accuracy: 0.9428 - loss: 0.7041 - mean_iou: 0.4568
Test Loss: 0.7041 | Test Acc: 0.9428 | Test IoU: 0.4568


**17th April**

In [4]:
# # -----------------------------
# # 1) Convert YOLOv11 polygons → PNG masks
# # -----------------------------
# import os
# import cv2
# import numpy as np
# from glob import glob
# from tqdm import tqdm

# # adjust to your Roboflow‐downloaded base
# base = dataset.location   # e.g. "/content/Microfocus-3"
# IMG_SIZE = (256, 256)     # width, height

# def yolo11_poly_to_mask(img_path: str, label_path: str, img_size=IMG_SIZE):
#     """
#     Read a YOLOv11 segmentation .txt and rasterize the polygon to a binary mask.
#     """
#     w, h = img_size
#     mask = np.zeros((h, w), dtype=np.uint8)

#     if not os.path.exists(label_path):
#         return mask

#     with open(label_path) as f:
#         for line in f:
#             parts = list(map(float, line.strip().split()))
#             # first entry is class id
#             cls = parts[0]
#             coords = parts[1:]
#             # coords should be even length
#             pts = np.array(coords).reshape(-1, 2)
#             # denormalize
#             pts[:, 0] = pts[:, 0] * w
#             pts[:, 1] = pts[:, 1] * h
#             pts = pts.astype(np.int32)
#             # fill polygon
#             cv2.fillPoly(mask, [pts], 255)

#     return mask

# def create_masks_for_split(split: str):
#     img_dir  = os.path.join(base, split, "images")
#     lbl_dir  = os.path.join(base, split, "labels")
#     msk_dir  = os.path.join(base, split, "masks")
#     os.makedirs(msk_dir, exist_ok=True)

#     for img_path in tqdm(glob(os.path.join(img_dir, "*.jpg")), desc=f"{split} → masks"):
#         fn      = os.path.basename(img_path).rsplit(".", 1)[0]
#         lbl_fn  = os.path.join(lbl_dir, fn + ".txt")
#         mask    = yolo11_poly_to_mask(img_path, lbl_fn, IMG_SIZE)
#         cv2.imwrite(os.path.join(msk_dir, fn + ".png"), mask)

# # regenerate masks
# create_masks_for_split("train")
# create_masks_for_split("valid")
# create_masks_for_split("test")  # optional


test → masks: 100%|██████████| 50/50 [00:00<00:00, 1758.12it/s]


In [5]:
# import tensorflow as tf
# from glob import glob

# BATCH_SIZE = 8
# AUTOTUNE   = tf.data.AUTOTUNE

# def load_image_and_mask(img_path, mask_path):
#     img = tf.io.read_file(img_path)
#     img = tf.image.decode_jpeg(img, channels=3)
#     img = tf.image.resize(img, IMG_SIZE) / 255.0

#     mask = tf.io.read_file(mask_path)
#     mask = tf.image.decode_png(mask, channels=1)
#     mask = tf.image.resize(mask, IMG_SIZE,
#                            method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
#     mask = tf.cast(mask > 127, tf.float32)
#     return img, mask

# def make_ds(split: str):
#     img_dir  = os.path.join(base, split, "images")
#     mask_dir = os.path.join(base, split, "masks")

#     imgs  = sorted(glob(os.path.join(img_dir, "*.jpg")))
#     masks = [os.path.join(mask_dir, os.path.basename(p).replace(".jpg", ".png"))
#              for p in imgs]

#     ds = tf.data.Dataset.from_tensor_slices((
#         tf.constant(imgs, dtype=tf.string),
#         tf.constant(masks, dtype=tf.string)
#     ))
#     ds = ds.map(load_image_and_mask, num_parallel_calls=AUTOTUNE)
#     ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
#     return ds

# train_ds = make_ds("train")
# val_ds   = make_ds("valid")
# test_ds = make_ds("test")

In [6]:
# import tensorflow as tf

# # -----------------------------
# # STEP 5: Build ResNet‑Attention U‑Net (fixed upsampling)
# # -----------------------------
# def conv_block(x, filters):
#     x = tf.keras.layers.Conv2D(filters, 3, padding="same", activation="relu")(x)
#     x = tf.keras.layers.BatchNormalization()(x)
#     x = tf.keras.layers.Conv2D(filters, 3, padding="same", activation="relu")(x)
#     x = tf.keras.layers.BatchNormalization()(x)
#     return x

# def attention_block(skip, gate, inter_channels):
#     # 1×1 conv on skip and gate
#     theta_x = tf.keras.layers.Conv2D(inter_channels, 1, padding="same")(skip)
#     phi_g   = tf.keras.layers.Conv2D(inter_channels, 1, padding="same")(gate)

#     # Resize gate to match skip shape
#     gate_shape = tf.keras.backend.int_shape(skip)[1:3]
#     phi_g_up   = tf.keras.layers.UpSampling2D(size=(
#         gate_shape[0] // tf.keras.backend.int_shape(gate)[1],
#         gate_shape[1] // tf.keras.backend.int_shape(gate)[2]
#     ), interpolation="bilinear")(phi_g)

#     # Attention mechanism
#     add     = tf.keras.layers.Add()([theta_x, phi_g_up])
#     act     = tf.keras.layers.Activation("relu")(add)
#     psi     = tf.keras.layers.Conv2D(1, 1, padding="same", activation="sigmoid")(act)

#     # Multiply with skip connection
#     attn    = tf.keras.layers.Multiply()([skip, psi])
#     return attn


# def build_res_attention_unet(input_shape=(256,256,3)):
#     inputs     = tf.keras.Input(shape=input_shape)
#     base_model = tf.keras.applications.ResNet50(
#         include_top=False, weights="imagenet", input_tensor=inputs
#     )
#     # capture skip features
#     skips = [
#         base_model.get_layer(name).output
#         for name in ["conv1_relu",
#                      "conv2_block3_out",
#                      "conv3_block4_out",
#                      "conv4_block6_out"]
#     ]
#     # bottleneck
#     x = base_model.get_layer("conv5_block3_out").output

#     # decoder filters (match encoder)
#     filters = [512, 256, 128, 64]
#     for i, skip in enumerate(reversed(skips)):
#         x = tf.keras.layers.UpSampling2D(size=(2,2), interpolation="bilinear")(x)
#         attn = attention_block(skip, x, inter_channels=filters[i]//2)
#         x = tf.keras.layers.Concatenate()([x, attn])
#         x = conv_block(x, filters[i])

#     # final up to original resolution
#     x = tf.keras.layers.UpSampling2D(size=(2,2), interpolation="bilinear")(x)
#     x = conv_block(x, 32)
#     outputs = tf.keras.layers.Conv2D(1, 1, activation="sigmoid")(x)

#     return tf.keras.Model(inputs, outputs, name="ResNet_Att_UNet")

# # Instantiate & compile
# model = build_res_attention_unet((IMG_SIZE[1], IMG_SIZE[0], 3))
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(1e-4),
#     loss="binary_crossentropy",
#     metrics=[
#         "accuracy",
#         tf.keras.metrics.MeanIoU(num_classes=2, name="mean_iou")
#     ]
# )
# model.summary()


Model: "ResNet_Att_UNet"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 256, 256, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 262, 262, 3)    │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 128, 128, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 128, 128, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 128, 128, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 130, 130, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 64, 64, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 64, 64, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 64, 64, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 64, 64, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 64, 64, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 64, 64, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 64, 64, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 64, 64, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 64, 64, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 44,932,101 (171.40 MB)

 Trainable params: 44,875,013 (171.18 MB)

 Non-trainable params: 57,088 (223.00 KB)

In [7]:
# # STEP 7: Callbacks
# # -----------------------------
# callbacks = [
#     # save full model in .keras format
#     tf.keras.callbacks.ModelCheckpoint(
#         "resattunet_best.keras",
#         monitor="val_loss",
#         save_best_only=True
#     ),
#     tf.keras.callbacks.ReduceLROnPlateau(
#         monitor="val_loss",
#         factor=0.5,
#         patience=30,
#         verbose=1
#     ),
#     tf.keras.callbacks.EarlyStopping(
#         monitor="val_loss",
#         patience=70,
#         restore_best_weights=True
#     )
# ]

In [8]:
# # -----------------------------
# # STEP 8: Train
# # -----------------------------
# EPOCHS = 100
# history = model.fit(
#     train_ds,
#     validation_data=val_ds,
#     epochs=EPOCHS,
#     callbacks=callbacks,
#     verbose=True
# )


Epoch 1/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 171s 795ms/step - accuracy: 0.7413 - loss: 0.6334 - mean_iou: 0.3815 - val_accuracy: 0.5308 - val_loss: 0.8729 - val_mean_iou: 0.4045 - learning_rate: 1.0000e-04
Epoch 2/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 50s 463ms/step - accuracy: 0.8434 - loss: 0.5156 - mean_iou: 0.3821 - val_accuracy: 0.7412 - val_loss: 0.5739 - val_mean_iou: 0.4045 - learning_rate: 1.0000e-04
Epoch 3/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 47s 442ms/step - accuracy: 0.8899 - loss: 0.4283 - mean_iou: 0.3894 - val_accuracy: 0.8081 - val_loss: 0.5178 - val_mean_iou: 0.4045 - learning_rate: 1.0000e-04
Epoch 4/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 45s 419ms/step - accuracy: 0.8976 - loss: 0.4078 - mean_iou: 0.3923 - val_accuracy: 0.7104 - val_loss: 0.6388 - val_mean_iou: 0.4045 - learning_rate: 1.0000e-04
Epoch 5/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 44s 414ms/step - accuracy: 0.9062 - loss: 0.3765 - mean_iou: 0.3910 - val_accuracy: 0.5652 - val_loss: 1.2014 - val_mean_iou: 0.4045 - learning_rate: 